## LOADING AND PREPARING THE DATA ##

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
#open the training file and assign to variable text
with open("/kaggle/input/the-bards-best-a-character-modeling-dataset/train.csv", "r", encoding="utf-8") as f:
    train_text = f.read()
    
with open("/kaggle/input/the-bards-best-a-character-modeling-dataset/validation.csv", "r", encoding="utf-8") as t:
    val_text = t.read()

In [3]:
print("The length of the file is ", len(train_text))

The length of the file is  1003862


In [4]:
print(train_text[:500])

text
"First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounte


## TOKENIZATION AND ENCODING/DECODING ##

#### Tokenizing the characters ####

In [5]:
chars = sorted(list(set(train_text)))#assorted list of characters
vocab_size = len(chars)
print("".join(chars))#print characters
print(vocab_size)#number of characters


 !"$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
66


#### Decoding and encoding functions ####

In [6]:

stoi = { ch:i for i,ch in enumerate(chars)}#dictionary for string to integer
itos = { i:ch for i,ch in enumerate(chars)}#dictionary for integer to string
encode = lambda s: [stoi[c] for c in s]#function for string -> int
decode = lambda l: "".join([itos[i] for i in l])#function for int -> string

#EXAMPLE
print(encode("transformer"))
print(decode(encode("transformer")))

[59, 57, 40, 53, 58, 45, 54, 57, 52, 44, 57]
transformer


#### Creating tensors out of text sequences ####

In [7]:
train_data = torch.tensor(encode(train_text), dtype= torch.long)#tensor of all encoded training data 
val_data = torch.tensor(encode(val_text), dtype= torch.long)#same for validation data

#VISUALIZING AN EXAMPLE
print(train_data.shape, train_data.dtype)
print(train_data[:100])

torch.Size([1003862]) torch.int64
tensor([59, 44, 63, 59,  0,  3, 19, 48, 57, 58, 59,  1, 16, 48, 59, 48, 65, 44,
        53, 11,  0, 15, 44, 45, 54, 57, 44,  1, 62, 44,  1, 55, 57, 54, 42, 44,
        44, 43,  1, 40, 53, 64,  1, 45, 60, 57, 59, 47, 44, 57,  7,  1, 47, 44,
        40, 57,  1, 52, 44,  1, 58, 55, 44, 40, 50,  9,  0,  0, 14, 51, 51, 11,
         0, 32, 55, 44, 40, 50,  7,  1, 58, 55, 44, 40, 50,  9,  0,  0, 19, 48,
        57, 58, 59,  1, 16, 48, 59, 48, 65, 44])


# THE MODEL #

#### Defining some Variables ####

In [8]:

block_size = 8 #the size of the "chunks" passed through the transformer
batch_size = 4 #how many blocks will be processed at a time
n_embd = 32 #size of vector embeddings for each token
eval_iters=500
eval_interval=100
head_size = 16
n_head = 4 #how many sections we divide each embedding, every head has 384/6 = 64 dimension
n_layer = 6 #how many blocks
dropout = 0.2 #20% nodes will drop to zero
torch.manual_seed(598) #manually set random number


#### Visualizing a sequence example ####

In [9]:
x= train_data[:block_size] #x= numbers in position 0-7 - input
y= train_data[1:block_size+1] #y= numbers in position 1-8 - target
print(str(x))
print(str(y))

#for t in range(block_size): #visualize context and target at each position
    #context = x[:t+1]#all characters up to t (including t) in x list
   # target = y[t]#t'th character in y list 
    #print(f"when input is {context} the target is:{target}")

tensor([59, 44, 63, 59,  0,  3, 19, 48])
tensor([44, 63, 59,  0,  3, 19, 48, 57])


### Getting samples from our dataset ### 
Getting random blocks/sequences from our dataset and stacking them into tensors

In [10]:
def get_batch(split):
    data = train_data if split == "train" else val_data #validation or training?
    ix = torch.randint(len(data) - block_size, (batch_size,))#generating 4 different random numbers between 0 and len(data)-8 
    x = torch.stack([data[i:i+block_size] for i in ix])#get 8 characters after each of 4 random numbers
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])#same now, shifted one unit right 
    return x, y#returning both tensors as a tuple


#VISUALIZING AN EXAMPLE

xb, yb = get_batch('train')#rename x and y as xb and yb for the sake of clarity
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[ 1, 52, 44,  1, 51, 44, 40, 61],
        [43,  1, 41, 44, 45, 54, 57, 44],
        [ 1, 40, 53, 43,  1, 51, 54, 40],
        [57,  1, 52, 64,  1, 41, 54, 60]])
targets:
torch.Size([4, 8])
tensor([[52, 44,  1, 51, 44, 40, 61, 44],
        [ 1, 41, 44, 45, 54, 57, 44,  9],
        [40, 53, 43,  1, 51, 54, 40, 59],
        [ 1, 52, 64,  1, 41, 54, 60, 53]])


#### Loss function ####
Estimated loss: average predicted loss for training and validating data 

In [11]:
@torch.no_grad() #stop calculation of gradients 
def estimate_loss():
    out = {}  #initialize an empty dictionary
    model.eval() #no changes in the model (only evaluation)
    for split in ["train","val"]: #loop over training and validation
        losses = torch.zeros(eval_iters) #new tensor of zeros with length eval_iters
        for k in range(eval_iters):#go through k number of batches
            X, Y = get_batch(split) #batch of x and y sequences from val and train
            logits, loss = model(X,Y)#run model and calculate loss value
            losses[k] = loss.item()#add loss to losses
        out[split] = losses.mean()#calculate mean
    model.train()#back to training mode
    return out


### Single head self attention ###

In [12]:
class Head(nn.Module):
    
    def __init__(self,head_size):
        super().__init__()
        #multiplying the tensors by some fixed weights
        self.key = nn.Linear(n_embd, head_size, bias=False) # (B,T,C) --> (B,T,16)
        self.query = nn.Linear(n_embd, head_size, bias=False) # (B,T,C) --> (B,T,16)
        self.value = nn.Linear(n_embd, head_size, bias=False) # (B,T,C) --> (B,T,16)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size))) 
        #it is a tensor called trill of size block_size that has ones in the bottom triangle and zeros in the top
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):#all tokens produce a key and a query 
        B,T,C = x.shape
        k = self.key(x) #(B,T,32) (each key and query is made of 32 numbers)
        q = self.query(x) #(B,T,32)
        wei = q@k.transpose(-2,-1) * C**-0.5 # switch the last two (B,T,16) dimensions of k (B,T,16) @ (B,16,T) --> (B,T,T)
        wei = wei.masked_fill(self.tril[:T,:T]== 0, float("-inf"))
        #tensor with bottom triangle of zeros, and upper of -inf (ignoring those tokens)
        wei=F.softmax(wei, dim=-1) #-1 dimension = horizontally
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei@v #weight the value vectors of all the tokens 
        return out

### Multi-Head Self-Attention ###

In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__() #initialize nn.Module
        #head size is 8 and num_heads is 4 so stacking them would result in 32 (our n_embd)
        self.heads = nn.ModuleList([Head(head_size) for _ in range (num_heads)]) 
        #create a list called heads which stores the output embeddings of the Head class for each of the heads
        self.proj = nn.Linear(n_embd, n_embd) #linear transformation of the embeddings 
    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) #x is the sequences of tokens with their corresponding embeddings
        out = self.proj(out) #pass output through linear layer
        return out #put all the outputs from the list into a tensor
    
    #Ouput = tensor (B,T, embeddings of all the heads stacked together) which represents the
    #context of the input

### Feedforward ###
Feedforward is a simple layer to add computation to the model (make the relation betweek k, q and v more complex). Adds **non-linearity**.
Allow the model to learn more complex and **abstract representations** of the sequence, to improve the model's ability to understand and generate text.

In [14]:
#all tokens do independently 
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__() #call the nn.Module
        self.net = nn.Sequential( #sequential pytorch module 
            nn.Linear(n_embd, 4* n_embd), #linear NN layer in which we add a 4x bigger layer
            nn.ReLU(), #ReLU activation layer (all - numbers become 0)
            nn.Linear(4* n_embd, n_embd), #back to original size
            nn.Dropout(dropout), 
            #randomly set the weight of some neurons to zero to avoid overfitting.
        )
    
    def forward(self,x):
        return self.net(x)

### Decoder Block ###

In [15]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head # 32 dimension/4 heads = (embedding dimension per head)
        #communication
        self.sa= MultiHeadAttention(n_head, head_size)
        #computation
        self.ffwd = FeedForward(n_embd)
        #normalizing each embedding so that the embedding of each token (32 numbers) has mean 0 SD 1
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self,x):
        #applying skip/residual connections to come back to the original embeddings
        x = x + self.sa(self.ln1(x)) 
        x = x + self.ffwd(self.ln2(x))
        return x 

### Putting it all together ###

In [16]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self):#initializing the model
        super().__init__() #initializing nn.Module superclass
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        #map each token to a dense vector (layer of neurons)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        #each position will get its own embedding vector
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range (n_layer)]) 
        #pass sequence through all blocks sequentially
        self.ln_f = nn.LayerNorm(n_embd) #normalize the embeddings mean 0 SD 1 
        self.lm_head = nn.Linear(n_embd, vocab_size)

    #index and target are both tensors of dimension (B,T)
    def forward(self,idx,targets=None):
        B,T = idx.shape
        tok_emb = self.token_embedding_table(idx) # converting character index (B,T) -> embedding (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T)) 
        #torch.arange() assigns each position to an int
        x = tok_emb + pos_emb#add the pos embedding to every batch of the token embedding
        x = self.blocks(x) #apply the self attention
        x = self.ln_f(x)
        logits = self.lm_head(x) # embedding -> logits (B,T, vocab_size)
        
        if targets is None: #if there are no targets, no loss is calcualted
            loss = None
        else: 
            B,T,C = logits.shape #Reshaping our tensors to fit requirements for cross entropy
            logits = logits.view(B*T, C) #stacking the logits tensors (reduce to 2 dimensions)
            targets = targets.view(B*T) #stacking the targets tensors (reduce to 1 dimension)
            loss = F.cross_entropy(logits, targets) #calculating loss
        
        return(logits, loss)
    
    def generate(self, idx, max_new_tokens):#takes in the index of tensors 4x8 (BxT)
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:] #crops the context to be max the size of block size
            logits,loss = self(idx_cond) #make the predictions for the next character and ignore the loss
            logits = logits[:,-1,:] #remove the last character (last letter of each sequence)
            probs = F.softmax(logits, dim=-1) #pass the logits through the softmax
            idx_next = torch.multinomial(probs,num_samples=1) #perform weighted prediction
            idx = torch.cat((idx, idx_next), dim=1) #add predicted characters to the sequence and change t to t+1
        return idx
    
model = BigramLanguageModel()
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 66])
tensor(4.3175, grad_fn=<NllLossBackward0>)


In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)#adam is an advanced optimizer

# TRAINING #

In [18]:
batch_size = 32 
for steps in range(5000):
    
    if steps % eval_interval ==0: #sometimes calculate the estimated loss
        losses = estimate_loss() #run estimate_loss() function and store in variable losses 
        print(f'step {steps}: train loss {losses["train"]:.4f}, val loss {losses["val"]:.4f}')
        #print losses rounded to 4 decimal places
    
    xb, yb = get_batch("train") #get data from training data using our function
    logits,loss = model(xb,yb) #instance of our Bigram Language model
    optimizer.zero_grad(set_to_none=True) #previous gradients are set to zero
    loss.backward() #evaluating loss
    optimizer.step() #update parameters

step 0: train loss 4.3507, val loss 4.3569
step 100: train loss 3.3863, val loss 3.4042
step 200: train loss 3.1229, val loss 3.1543
step 300: train loss 2.9082, val loss 2.9353
step 400: train loss 2.7825, val loss 2.8033
step 500: train loss 2.7082, val loss 2.7213
step 600: train loss 2.6465, val loss 2.6592
step 700: train loss 2.5960, val loss 2.6034
step 800: train loss 2.5593, val loss 2.5723
step 900: train loss 2.5263, val loss 2.5339
step 1000: train loss 2.4923, val loss 2.5159
step 1100: train loss 2.4779, val loss 2.4875
step 1200: train loss 2.4561, val loss 2.4673
step 1300: train loss 2.4465, val loss 2.4491
step 1400: train loss 2.4218, val loss 2.4382
step 1500: train loss 2.4102, val loss 2.4214
step 1600: train loss 2.4009, val loss 2.4056
step 1700: train loss 2.3793, val loss 2.3967
step 1800: train loss 2.3691, val loss 2.3859
step 1900: train loss 2.3584, val loss 2.3748
step 2000: train loss 2.3521, val loss 2.3784
step 2100: train loss 2.3404, val loss 2.3503


In [19]:

#print(decode(model.generate(context,idx=369,max_new_tokens=2000)[0].tolist()))
#passing a context of zero
#tensor shape (1, 1) filled with 0 = starting token
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))




I and, anmpoly buth sownerere fuersearearet miong arounsse tilk
So thens Hosptreal this sheat go awelen aswing hir ham dis;son ha?
Nows GFough to le fits wich ther.
Hust PEd youse my kiDed
Un,,
As wherdy lIUCfon dee saterd is farded, prot froomim thou een towno le ang thay:
Firt, theeere a of you our detdang ted a this him Vimaty ae fiein thith ath you anlt be sownchat.

KEd cho fut nour mur art i
